## Intro
Information about this experiment can be found in the [markdown](./2024-10-21-try-llm-lowrop-holecleaning-packoff-with-debug.md).

## Set up environment

In [16]:
from dotenv import load_dotenv

_ = load_dotenv(override=True)

In [17]:
import os
from common.context import LLMTagPredictionContext

DEBUG = True
EXPERIMENT_ID = "2024-10-21-try-llm-lowrop-debug"
RUN_ID = "1-lowrop-gpt-4o-mini"

CONTEXT = LLMTagPredictionContext(
    description="Try asking LLM to assess all tags at once. Give examples.",
    experiment_id=EXPERIMENT_ID,
    run_id=RUN_ID,
    tags_in_scope=sorted(
        [
            "lowrop"
        ]
    ),
    llm_model=os.environ["AZURE_OPENAI_DEPLOYMENT_ID"],
    with_notags=True,
)

## Fetch datasets

In [18]:
from common.datasets import load_input_dataset

dataset_df = load_input_dataset(
    "reviewed_distributed_ddr_v3.csv",
    columns_to_convert_to_sets=["tags", "Reviewed tags"],
)
dataset_df

,reviewed_distributed_ddr_v2.csv,Text,phase,code,subCode,tags,Are tags correct?,Reviewed tags,Comments
0,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN
1,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN
2,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN
3,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN
4,a1f86f80-135e-458b-aafc-3af30d2476f2_main_e6c0...,Laid down cement head. Moved rig to J-3 slot....,SURF,P,LD,{shallowwater},YES,{shallowwater},NaN
...,...,...,...,...,...,...,...,...,...
1437,a1f86f80-135e-458b-aafc-3af30d2476f2_main_7965...,Meeting with onshore forward plan.\r\nMeanwhil...,INTERV,N,SAFETY,{},YES,{},NaN
1438,a1f86f80-135e-458b-aafc-3af30d2476f2_main_fc63...,Drilled from 3903 m to 3921 m. - WOB = 6 - ...,RES1,P,DRL,{},YES,{},NaN
1439,a1f86f80-135e-458b-aafc-3af30d2476f2_main_12d5...,CIRC OUT FILL W/ 80 SPM/2000 PSI.,PROD1,C,NaN,{},YES,{},NaN
1440,a1f86f80-135e-458b-aafc-3af30d2476f2_main_644e...,"Drilled 12-1/4"" hole from 3916m to 3931m with...",INT2,P,DRLDIR,{},NO,"{lowrop, lostcirculation}",ROP 10m/hr.


## Apply the model

In [19]:
import pandas as pd
from common.llm import ask_openai

examples = dataset_df.sort_values(by="Text", key=lambda x: x.str.len())
examples = examples[examples["Text"].str.len() > 40]

examples = [
    examples[examples["Reviewed tags"].apply(lambda tags: tag in tags)][
        ["Text", "Reviewed tags"]
    ].head(2)
    for tag in CONTEXT.tags_in_scope
]

# Concatenate the examples into a single DataFrame
examples = pd.concat(examples, ignore_index=True)
# Convert 'Reviewed tags' to a sorted list
examples["Reviewed tags"] = examples["Reviewed tags"].apply(
    lambda tags: tuple(sorted(tags))
)
# Drop duplicate rows
examples = examples.drop_duplicates()
examples = examples.sort_values(by="Text")
examples

,Text,Reviewed tags
1,Drilled hard stringer at 1 m/hr ROP; 9-11 mT W...,"(lowrop,)"
0,"Drilled very hard stringer from 4115-16 m, 1 m...","(lowrop,)"


In [20]:
global DEBUG
SYSTEM_PROMPT = f"""
You will be given a description of a daily drilling report, and your task is to select which of these tags apply to this report.

Possible tags: {', '.join(CONTEXT.tags_in_scope)}

Respond only with the list of tags that apply to the report, separated by commas. If no tags apply, respond with `no tags`. Stick to the definition for the tag, do not make assumptions beyond the given data.

Definition of Low Rate of Penetration (lowrop):

The reasons could be:
- hard formation like stringers.
- Worn drill bit.
- Bad drill bit design.
- Drilling parameters not optimised.

Rate of Penetration (ROP, drilling speed) is considered lowrop if:
- ROP is below 10 m/hr or 10 m/hr.
Example: "ROP = 6 m/hr" → Tag as lowrop.
OR
- If stringers or hard formations are encountered while drilling, tag with lowrop regardless of ROP values.
Example 1: Drilled through harder formation in interval 246 m - 248 m → Tag as lowrop, as hard formations are encountered.
Example 2: Drilled stringer from 2770-2771 m, ROP is 20 m/hr.  → Tag as lowrop, as stringers are encountered, even if ROP is above 10 m/hr.
Example 3: DRLG STRINGERS -> These are drilling stringers, tag as lowrop.

Do not tag as lowrop if the reduction in ROP is intentional, regardless of value or presence of stringers.
Examples of intentional reductions include: Directional Control, Milling, Logging, Ballooning, Washing, Controlled Drilling, Data Gathering.
# Example 1: Drilled with reduced ROP to obtain data points → Do not tag as lowrop.
# Example 2: Continued to log OOH from 4206 m to 3333 m at 1.5 m/min → Do not tag as lowrop.
# Example 3: Observed 0,9 m3/hr ballooning rate. → Do not tag as lowrop.
# Example 4: Commenced milling → Do not tag as lowrop.
Conversion Rules:
- 1 m/min = 60 m/hr, always perform the conversion if needed.
- 1 m3/hr = 1 m/hr
- 1 mph = 1 m/hr
- 1 mhr = 1 m/hr
Important: Ensure the model performs comparisons. Example: If the ROP is 6, it is below 10. If the ROP is 27, it is above 10.
If no tags apply, respond with no tags.

Other situations that might occur during drilling: Lost Circulation, Hard Drilling, Wellbore Stability, Stuck Pipe, Well Control, Hole Cleaning, Tight Hole, Boulders, Shallow Gas, Shallow Water, Wellbore Breathing, Pack-Off, Directional Control, High ROP, Downhole Equipment Failure, Wait, Surface Equipment Failure.These situations may or may not occur at the same time with Low Rate of Penetreation.
"""


# The reasons could be:
# - hard formation like stringers.
# - Worn drill bit.
# - Bad drill bit design.
# - Drilling parameters not optimised.
# Additional rule for tagging with lowrop: If ROP (instantaneous, inst.) is less than 10 m/hr, it is lowrop. Example: If ROP = 6 m/hr, it is lowrop because 6 is less than 10. If ROP is mentioned at least once with values below 10, it is lowrop.
# Additional rule for tagging with lowrop: If stringers or intervals **related to drilling** are mentioned, a low rate of penetretion is very likely to occur and can be tagged with lowrop even if the ROP is above 10.
# Example 1: Drilled through harder formation in interval 246 m - 248 m and 530 - 535 m. Tag with lowrop, even if ROP is above 10 m/hr.
# Example 2: Drilled stringer from 2770-2771 m, ROP is 20 m/hr. Tag with lowrop, even if ROP is above 10 m/hr.
# Conversion Rule: 1 m/min = 60 m/hr, and 1 mhr = 1 m/hr. 1 m3/hr = 1 m/hr. If no measures are indicated, assume m or meters.
# Important: Ensure the model performs comparisons. Example: If the ROP is 6, it is below 10, so it is lowrop. If the ROP is 27, it is above 10, so it is not lowrop.
# If the ROP is 10, it is below 100 and if stringers, intervals or hard formations are mentioned, it is lowrop.
# Intentional low rop should NOT be tagged as lowrop, regardless of value and presence of stringers.:
# - DIRECTIONAL CONTROL
# - milling
# - controlled drilling
# - WASHING or cleaning the hole
# - improve data gathering (number of measurements while drilling in an interval)
# - caution before drilling into a challenging formation.
# - logging
# - ballooning
# Example 1: Drilled with reduced ROP to obtain 3 data points per meter
# Example 2: Continued to log OOH from 4206 m to 3333 m at 1.5 m/min
# Example 3: Observed 0,9 m3/hr ballooning rate.
# Example 4: Commenced milling
# Other situations that might occur during drilling: Lost Circulation, Hard Drilling, Wellbore Stability, Stuck Pipe, Well Control, Hole Cleaning, Tight Hole, Boulders, Shallow Gas, Shallow Water, Wellbore Breathing, Pack-Off, Directional Control, High ROP, Downhole Equipment Failure, Wait, Surface Equipment Failure.These situations may or may not occur at the same time with Low Rate of Penetreation.
# """

if not DEBUG:
    for idx, (_, row) in enumerate(examples.iterrows()):
        SYSTEM_PROMPT += f"\n## Example report {idx}\nText: {row['Text']}\n\n## Correct response\n{', '.join(row['Reviewed tags'])}\n"

if DEBUG:
    SYSTEM_PROMPT += "Before you provide the answer, explain the reason for selecting each tag or lack of it. The reason should be supplied at the end, following |debug| separator. Answer template: tag1, tag2, tag3|debug|reason1, reason2, reason3"

CONTEXT.llm_system_prompt = SYSTEM_PROMPT

print(SYSTEM_PROMPT)
print(len(SYSTEM_PROMPT))


You will be given a description of a daily drilling report, and your task is to select which of these tags apply to this report.

Possible tags: lowrop

Respond only with the list of tags that apply to the report, separated by commas. If no tags apply, respond with `no tags`. Stick to the definition for the tag, do not make assumptions beyond the given data.

Definition of Low Rate of Penetration (lowrop):

The reasons could be:
- hard formation like stringers.
- Worn drill bit.
- Bad drill bit design.
- Drilling parameters not optimised.

Rate of Penetration (ROP, drilling speed) is considered lowrop if:
- ROP is below 10 m/hr or 10 m/hr.
Example: "ROP = 6 m/hr" → Tag as lowrop.
OR
- If stringers or hard formations are encountered while drilling, tag with lowrop regardless of ROP values.
Example 1: Drilled through harder formation in interval 246 m - 248 m → Tag as lowrop, as hard formations are encountered.
Example 2: Drilled stringer from 2770-2771 m, ROP is 20 m/hr.  → Tag as lowr

In [21]:
TEMPERATURE = 0

CONTEXT.llm_temperature = TEMPERATURE

In [22]:
from concurrent.futures import ThreadPoolExecutor
import os
from tqdm.auto import tqdm


# Define a function to call ask_openai and get the predicted tags
def get_predicted_tags(text) -> tuple[set, str | None, str | None]:
    global DEBUG
    system_prompt = SYSTEM_PROMPT
   
    try:
        response = ask_openai(
            azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
            api_key=os.environ["AZURE_OPENAI_KEY"],
            api_version=os.environ["AZURE_OPENAI_API_VERSION"],
            deployment_name=os.environ["AZURE_OPENAI_DEPLOYMENT_ID"],
            system_prompt=system_prompt,
            prompt=text,
        )
    except Exception as e:
        if "content management policy. Please modify your prompt" in str(e):
            print(e)
            return [], None, None  # running into the content filter
        raise

    def normalize_tag(t):
        # sometimes model makes mistakes
        t = t.lower()
        if t.startswith("tags:"):
            t = t[len("tags:") :]
        t = t.strip().strip("()")
        return t
    
    # Debug mode logic: if enabled, append reasoning for each tag
    if DEBUG and "|debug|" in response:
        resonse_tags = response.split("|debug|")[0].strip().split(",")
        debug_info = response.split("|debug|")[1].strip()  # Extract reasoning part
        tags = set(normalize_tag(t) for t in resonse_tags).intersection(
        CONTEXT.tags_in_scope
    )
        return tags, response, debug_info
    else:
        tags = set(normalize_tag(t) for t in response.strip().split(",")).intersection(
        CONTEXT.tags_in_scope)
        return tags, response, None

assessed_df = dataset_df.copy()


def parallel_apply(df, func, num_threads: int):
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        results = list(tqdm(executor.map(func, df["Text"]), total=len(df)))
    return results


assessed_df[["Predicted", "CompleteAnswer", "Debug"]] = parallel_apply(
     assessed_df, get_predicted_tags, num_threads=2
 )

assessed_df

  0%|          | 0/1442 [00:00<?, ?it/s]

100%|██████████| 1442/1442 [06:37<00:00,  3.63it/s]


,reviewed_distributed_ddr_v2.csv,Text,phase,code,subCode,tags,Are tags correct?,Reviewed tags,Comments,Predicted,CompleteAnswer,Debug
0,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,{},no tags|debug|The report does not indicate a l...,The report does not indicate a low rate of pen...
1,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,{},no tags|debug|The report does not indicate a l...,The report does not indicate a low rate of pen...
2,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,{},no tags|debug|The report does not indicate a l...,The report does not indicate a low rate of pen...
3,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,{},no tags|debug|The report describes activities ...,The report describes activities related to cir...
4,a1f86f80-135e-458b-aafc-3af30d2476f2_main_e6c0...,Laid down cement head. Moved rig to J-3 slot....,SURF,P,LD,{shallowwater},YES,{shallowwater},NaN,{},no tags|debug|The report does not mention any ...,The report does not mention any low rate of pe...
...,...,...,...,...,...,...,...,...,...,...,...,...
1437,a1f86f80-135e-458b-aafc-3af30d2476f2_main_7965...,Meeting with onshore forward plan.\r\nMeanwhil...,INTERV,N,SAFETY,{},YES,{},NaN,{},no tags|debug|The report does not indicate any...,The report does not indicate any low rate of p...
1438,a1f86f80-135e-458b-aafc-3af30d2476f2_main_fc63...,Drilled from 3903 m to 3921 m. - WOB = 6 - ...,RES1,P,DRL,{},YES,{},NaN,{},"no tags|debug|The ROP is +/- 30 m/hr, which is...","The ROP is +/- 30 m/hr, which is above the thr..."
1439,a1f86f80-135e-458b-aafc-3af30d2476f2_main_12d5...,CIRC OUT FILL W/ 80 SPM/2000 PSI.,PROD1,C,NaN,{},YES,{},NaN,{},no tags|debug|There is no indication of a low ...,There is no indication of a low rate of penetr...
1440,a1f86f80-135e-458b-aafc-3af30d2476f2_main_644e...,"Drilled 12-1/4"" hole from 3916m to 3931m with...",INT2,P,DRLDIR,{},NO,"{lowrop, lostcirculation}",ROP 10m/hr.,{},"no tags|debug|The ROP is exactly 10 m/hr, whic...","The ROP is exactly 10 m/hr, which does not qua..."


In [23]:
assessed_df.head()

,reviewed_distributed_ddr_v2.csv,Text,phase,code,subCode,tags,Are tags correct?,Reviewed tags,Comments,Predicted,CompleteAnswer,Debug
0,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,{},no tags|debug|The report does not indicate a l...,The report does not indicate a low rate of pen...
1,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,{},no tags|debug|The report does not indicate a l...,The report does not indicate a low rate of pen...
2,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,{},no tags|debug|The report does not indicate a l...,The report does not indicate a low rate of pen...
3,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,{},no tags|debug|The report describes activities ...,The report describes activities related to cir...
4,a1f86f80-135e-458b-aafc-3af30d2476f2_main_e6c0...,Laid down cement head. Moved rig to J-3 slot....,SURF,P,LD,{shallowwater},YES,{shallowwater},NaN,{},no tags|debug|The report does not mention any ...,The report does not mention any low rate of pe...


In [24]:
# nothing to do, DDR tagging using regex rules is already applied to the dataset in this experiment
from typing import Iterable
from common.assessment import expand_tags

assessed_df = expand_tags(
    assessed_df,
    tags_in_scope=CONTEXT.tags_in_scope,
    ground_truth_tags_column="Reviewed tags",
    predicted_tags_column="Predicted",
)
assessed_df

,reviewed_distributed_ddr_v2.csv,Text,phase,code,subCode,tags,Are tags correct?,Reviewed tags,Comments,CompleteAnswer,Debug,expected__lowrop,actual__lowrop
0,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,no tags|debug|The report does not indicate a l...,The report does not indicate a low rate of pen...,False,False
1,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,no tags|debug|The report does not indicate a l...,The report does not indicate a low rate of pen...,False,False
2,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,no tags|debug|The report does not indicate a l...,The report does not indicate a low rate of pen...,False,False
3,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,no tags|debug|The report describes activities ...,The report describes activities related to cir...,False,False
4,a1f86f80-135e-458b-aafc-3af30d2476f2_main_e6c0...,Laid down cement head. Moved rig to J-3 slot....,SURF,P,LD,{shallowwater},YES,{shallowwater},NaN,no tags|debug|The report does not mention any ...,The report does not mention any low rate of pe...,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1437,a1f86f80-135e-458b-aafc-3af30d2476f2_main_7965...,Meeting with onshore forward plan.\r\nMeanwhil...,INTERV,N,SAFETY,{},YES,{},NaN,no tags|debug|The report does not indicate any...,The report does not indicate any low rate of p...,False,False
1438,a1f86f80-135e-458b-aafc-3af30d2476f2_main_fc63...,Drilled from 3903 m to 3921 m. - WOB = 6 - ...,RES1,P,DRL,{},YES,{},NaN,"no tags|debug|The ROP is +/- 30 m/hr, which is...","The ROP is +/- 30 m/hr, which is above the thr...",False,False
1439,a1f86f80-135e-458b-aafc-3af30d2476f2_main_12d5...,CIRC OUT FILL W/ 80 SPM/2000 PSI.,PROD1,C,NaN,{},YES,{},NaN,no tags|debug|There is no indication of a low ...,There is no indication of a low rate of penetr...,False,False
1440,a1f86f80-135e-458b-aafc-3af30d2476f2_main_644e...,"Drilled 12-1/4"" hole from 3916m to 3931m with...",INT2,P,DRLDIR,{},NO,"{lowrop, lostcirculation}",ROP 10m/hr.,"no tags|debug|The ROP is exactly 10 m/hr, whic...","The ROP is exactly 10 m/hr, which does not qua...",True,False


In [25]:
from common.datasets import save_assessed_dataset

save_assessed_dataset(
    assessed_df,
    context=CONTEXT,
    experiment_id=EXPERIMENT_ID,
    run_id=RUN_ID,
)

2_assessed_datasets\2024-10-21-try-llm-lowrop-debug-1-lowrop-gpt-4o-mini


## Evaluate predicted tags

In [26]:
from common.evaluation import TagMatchingEvaluator

evaluator = TagMatchingEvaluator(
    assessed_df=assessed_df,
    tags_in_scope=CONTEXT.tags_in_scope,
    with_notags=CONTEXT.with_notags,
)

In [27]:
evaluator.eval_individual_ddrs()

,reviewed_distributed_ddr_v2.csv,Text,phase,code,subCode,tags,Are tags correct?,Reviewed tags,Comments,CompleteAnswer,Debug,expected__lowrop,actual__lowrop,expected__notags,actual__notags,precision,recall,f1,true_positives
0,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,no tags|debug|The report does not indicate a l...,The report does not indicate a low rate of pen...,False,False,True,True,1.0,1.0,1.0,1
1,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,no tags|debug|The report does not indicate a l...,The report does not indicate a low rate of pen...,False,False,True,True,1.0,1.0,1.0,1
2,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,no tags|debug|The report does not indicate a l...,The report does not indicate a low rate of pen...,False,False,True,True,1.0,1.0,1.0,1
3,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,no tags|debug|The report describes activities ...,The report describes activities related to cir...,False,False,True,True,1.0,1.0,1.0,1
4,a1f86f80-135e-458b-aafc-3af30d2476f2_main_e6c0...,Laid down cement head. Moved rig to J-3 slot....,SURF,P,LD,{shallowwater},YES,{shallowwater},NaN,no tags|debug|The report does not mention any ...,The report does not mention any low rate of pe...,False,False,True,True,1.0,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1437,a1f86f80-135e-458b-aafc-3af30d2476f2_main_7965...,Meeting with onshore forward plan.\r\nMeanwhil...,INTERV,N,SAFETY,{},YES,{},NaN,no tags|debug|The report does not indicate any...,The report does not indicate any low rate of p...,False,False,True,True,1.0,1.0,1.0,1
1438,a1f86f80-135e-458b-aafc-3af30d2476f2_main_fc63...,Drilled from 3903 m to 3921 m. - WOB = 6 - ...,RES1,P,DRL,{},YES,{},NaN,"no tags|debug|The ROP is +/- 30 m/hr, which is...","The ROP is +/- 30 m/hr, which is above the thr...",False,False,True,True,1.0,1.0,1.0,1
1439,a1f86f80-135e-458b-aafc-3af30d2476f2_main_12d5...,CIRC OUT FILL W/ 80 SPM/2000 PSI.,PROD1,C,NaN,{},YES,{},NaN,no tags|debug|There is no indication of a low ...,There is no indication of a low rate of penetr...,False,False,True,True,1.0,1.0,1.0,1
1440,a1f86f80-135e-458b-aafc-3af30d2476f2_main_644e...,"Drilled 12-1/4"" hole from 3916m to 3931m with...",INT2,P,DRLDIR,{},NO,"{lowrop, lostcirculation}",ROP 10m/hr.,"no tags|debug|The ROP is exactly 10 m/hr, whic...","The ROP is exactly 10 m/hr, which does not qua...",True,False,False,True,0.0,0.0,0.0,0


In [28]:
evaluator.eval_per_tag()

,tag,precision,recall,f1,true_positives,positives_in_ground_truth,negatives_in_ground_truth
0,lowrop,0.780488,0.901408,0.836601,128,142,1300


In [29]:
evaluator.average_metrics()

,Type,precision,recall,f1
0,Average per DDR,0.965326,0.965326,0.965326
1,Average per Tag,0.780488,0.901408,0.836601


## Save evaluation report

In [30]:
from common.datasets import save_evaluation_report

save_evaluation_report(
    experiment_id=EXPERIMENT_ID,
    run_id=RUN_ID,
    dataset_df=dataset_df,
    assessed_df=assessed_df,
    evaluator=evaluator,
    context=CONTEXT,
)